<a href="https://colab.research.google.com/github/Xurri/ML_project/blob/main/08_04_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Фаза 2 • Неделя 8 • Четверг
## Нейронные сети
### 🔥 PyTorch: fine tuning

### Подготовка к работе

1. Загрузи этот ноутбук на Google Colab
2. Подгрузи архив [датасета](https://www.kaggle.com/datasets/ikobzev/architectural-heritage-elements-image64-dataset) в свое пространство и разархивируй его с помощью `unzip`.

In [1]:
! pip install kaggle

In [2]:

from google.colab import files

In [3]:
uploaded = files.upload()

for filename in uploaded.keys():
  print(f'Загружен файл: {filename}')

Saving archive.zip to archive.zip
Загружен файл: archive.zip


In [4]:
zip_file_name = list(uploaded.keys())[0]

In [5]:
import os
import zipfile

In [6]:
output_dir = '/content/архив'
os.makedirs(output_dir, exist_ok=True)

In [7]:
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
  zip_ref.extractall(output_dir)

1. Убедись, что структура папок соответствует задаче классификации (либо приведите ее к формату, указанному ниже):

        `train`
        
            - class_1
            - class_2
            - ...
            - class_n
            
        `valid`

            - class_1
            - class_2
            - ...
            - class_n

In [8]:
!pip -qq install torchutils

2. Создай `DataLoader` в для обучающей и валидационных выборок. Примените аугментации к изображениям.

In [9]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms as T
from torchvision import io

from torchvision.transforms import RandomHorizontalFlip, RandomRotation, RandomVerticalFlip, ColorJitter, RandomCrop


import torchutils as tu
import json
import numpy as np
import matplotlib.pyplot as plt

In [10]:
print(torch.__version__, torchvision.__version__)

2.4.1+cu121 0.19.1+cu121


In [11]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

cuda


In [12]:
# определение пути
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'test')

In [13]:
train_dir, val_dir


('/content/архив/train', '/content/архив/test')

In [14]:
transform = T.Compose([
                       T.ColorJitter(),
                       T.ToTensor(),
                       T.Resize((224, 224))
                       ])

# Создание датасетов
train_dataset = torchvision.datasets.ImageFolder(train_dir, transform=transform)
val_dataset = torchvision.datasets.ImageFolder(val_dir, transform=transform)

# Создание DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

In [15]:
train_loader

3. Создай экземпляр предобученной модели: [torchvision models](https://pytorch.org/vision/stable/models.html). Можно взять любую модель для baseline, а дальше попробовать что-то более сложное.

In [16]:
from torchvision.models import resnet18, ResNet18_Weights
# Создание экземпляра модели ResNet
model = torchvision.models.resnet18(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 107MB/s]


In [17]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

5. Замени выходной слой форматом, который подходит под задачу: бинарная или многоклассовая классификация.

In [18]:
# Проверяем последний слой
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [19]:
# Изменение последнего слоя для задачи многоклассовой классификации
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 10)  # 10 классов

# Проверяем последний слой
print(model.fc)

Linear(in_features=512, out_features=10, bias=True)


6. Обучи модель (только последний слой!) и зафиксируй метрику на валидационной части выборки.

In [20]:
# переносим модель на девайс
model = model.to(DEVICE)

In [21]:
# Оптимизатор и функция потерь
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [22]:
# Проверяем, какие параметры обучаемые
for param in model.parameters():
    print(param.requires_grad)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [23]:
for param in model.parameters():
    param.requires_grad = False

In [24]:
for param in model.parameters():
    print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [25]:
# «Размораживаем» те веса, которые будем
# обучать
model.fc.weight.requires_grad = True
model.fc.bias.requires_grad = True

In [26]:
# Проверяем, True должно встретиться только в самом конце
for param in model.parameters():
    print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True


In [27]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

if DEVICE == 'cuda':
    print("Используем GPU")
else:
    print("GPU не доступен, используем CPU")


Используем GPU


In [30]:
# Цикл обучения
epochs = 15
train_epoch_acc = []
train_epoch_losses = []
valid_epoch_losses = []
valid_epoch_acc =[]
for epoch in range(epochs):
    model.train()
    loss_batch = []
    acc_batch  = []

    for images, labels in train_loader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE).long() # Преобразуем метки в тип Long

        preds = model(images).squeeze(-1)
        loss = criterion(preds, labels)
        loss_batch.append(loss.item())
        accuracy = (preds.argmax(dim=1) == labels).cpu().numpy().mean()
        acc_batch.append(accuracy)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_epoch_losses.append(np.mean(loss_batch))
    train_epoch_acc.append(np.mean(acc_batch))

    model.eval()
    loss_batch = []
    acc_batch  = []
    for images, labels in val_loader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE).long() # Преобразуем метки в тип Long

        with torch.no_grad():
            preds = model(images).squeeze(-1)

        loss = criterion(preds, labels)
        loss_batch.append(loss.item())

        accuracy = (preds.argmax(dim=1) == labels).cpu().numpy().mean()
        acc_batch.append(accuracy)

    valid_epoch_losses.append(np.mean(loss_batch))
    valid_epoch_acc.append(np.mean(acc_batch))

    print(f'Epoch: {epoch}  loss_train: {train_epoch_losses[-1]:.3f}, loss_valid: {valid_epoch_losses[-1]:.3f}')
    print(f'\t  metrics_train: {train_epoch_acc[-1]:.3f}, metrics_valid: {valid_epoch_acc[-1]:.3f}')

Epoch: 0  loss_train: 0.261, loss_valid: 0.314
	  metrics_train: 0.918, metrics_valid: 0.894
Epoch: 1  loss_train: 0.255, loss_valid: 0.306
	  metrics_train: 0.916, metrics_valid: 0.886
Epoch: 2  loss_train: 0.236, loss_valid: 0.284
	  metrics_train: 0.924, metrics_valid: 0.902
Epoch: 3  loss_train: 0.240, loss_valid: 0.283
	  metrics_train: 0.921, metrics_valid: 0.905
Epoch: 4  loss_train: 0.226, loss_valid: 0.279
	  metrics_train: 0.924, metrics_valid: 0.900
Epoch: 5  loss_train: 0.218, loss_valid: 0.296
	  metrics_train: 0.927, metrics_valid: 0.898
Epoch: 6  loss_train: 0.217, loss_valid: 0.297
	  metrics_train: 0.928, metrics_valid: 0.896
Epoch: 7  loss_train: 0.210, loss_valid: 0.278
	  metrics_train: 0.929, metrics_valid: 0.900
Epoch: 8  loss_train: 0.208, loss_valid: 0.301
	  metrics_train: 0.928, metrics_valid: 0.895
Epoch: 9  loss_train: 0.196, loss_valid: 0.302
	  metrics_train: 0.934, metrics_valid: 0.902
Epoch: 10  loss_train: 0.195, loss_valid: 0.289
	  metrics_train: 0.93

7. Распечатай изображение из тестовой выборки и подпиши класс картинки, предсказанный моделью.

In [31]:
idx2class= {j: i for i, j in train_dataset.class_to_idx.items()}
idx2class

{0: 'altar',
 1: 'apse',
 2: 'bell_tower',
 3: 'column',
 4: 'dome(inner)',
 5: 'dome(outer)',
 6: 'flying_buttress',
 7: 'gargoyle',
 8: 'stained_glass',
 9: 'vault'}

In [35]:
# подготовим функцию resize для картинок
resize = T.Resize((224, 224))

img = resize(io.read_image('image.jpg')/255)
plt.imshow(torch.permute(img, (1, 2, 0)))
with torch.inference_mode():
    pred_class = torch.sigmoid(model(img.unsqueeze(0).to(DEVICE))).round().item()
plt.axis('off')
plt.title(idx2class[pred_class]);

RuntimeError: [Errno 2] No such file or directory: 'image.jpg'

8. "Разморозь" несоклько слоев базовой модели и вновь обучи ее. Зафиксируй метрику качества для модели. О том, как сделать unfreeze нескольких слоев сразу, а не только последнего, можно почитать тут: [discuss.pytorch.org](https://discuss.pytorch.org/t/how-the-pytorch-freeze-network-in-some-layers-only-the-rest-of-the-training/7088/3)

In [ ]:
# code

9. Сравни качество двух моделей: предобученной с замененным выходным слоем и дообученной с несколькими размороженными слоями.

In [ ]:
# code

<img src="https://icons.iconarchive.com/icons/icons8/windows-8/256/Programming-Github-icon.png" width=32 /> Сохрани файл для __github__ и распечатай результат команды `!git status` в ячейке ниже.

In [ ]:
# code

10. Сохрани модель (пример можно посмотреть [тут](../../learning/aux/model_saving.ipynb) или в [документации](https://pytorch.org/tutorials/beginner/saving_loading_models.html))

In [ ]:
# code

11. Реализуй функцию, которая на вход принимает путь к файлу, а в ответ возвращает класс объекта.

In [ ]:
def get_prediction(path: str) -> str:
    pass

12. Сохрани ноутбук на github